In [ ]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [ ]:
import requests

# url = "https://hf-mirror.com"  # 替换为你想要检查的网址
url = "https://huggingface.co/"

try:
    response = requests.get(url)
    response.raise_for_status()  # 如果响应状态码不是 200，则抛出 HTTPError 异常
    print("网站访问正常")
except requests.exceptions.HTTPError as errh:
    print ("Http Error:", errh)
except requests.exceptions.ConnectionError as errc:
    print ("Error Connecting:", errc)
except requests.exceptions.Timeout as errt:
    print ("Timeout Error:", errt)
except requests.exceptions.RequestException as err:
    print ("OOps: Something Else", err)

# 加载 model 和 tokenizer

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForPreTraining
import os
import torch
# 设置HF_HOME环境变量
# os.environ['HF_HOME'] = "/root/autodl-fs/pre-trained-models/"
# os.environ['HF_ENDPOINT'] = "https://hf-mirror.com"
device = "cuda" # the device to load the model onto


In [ ]:

# 加载模型和分词器
model_name = "Qwen/Qwen2-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, torch_dtype=torch.float16, device_map=device, cache_dir="/root/autodl-fs/pre-trained-models/hub/")
model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device, cache_dir="/root/autodl-fs/pre-trained-models/hub/")

# model.cuda()
# model.device

# chat 模式

In [ ]:


prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
print(text)
print(model_inputs)
print(len(generated_ids))
print(generated_ids[0].shape)
print(response)

# 一般模式

In [ ]:
choices = ["A", "B", "C", "D"]
choice_ids = [tokenizer(choice)["input_ids"][0] for choice in choices]
print(choice_ids)

In [ ]:
logits = model(model_inputs.input_ids)["logits"]
print(logits)
print(logits.shape)

In [ ]:
last_token_logits = logits[:, -1, :]
print(last_token_logits)
print(last_token_logits.shape)

In [ ]:
choice_logits = last_token_logits[:, choice_ids].detach().cpu().numpy()
print(choice_logits)

In [ ]:
import numpy as np
def softmax(x):
    z = x - max(x)
    numerator = np.exp(z)
    denominator = np.sum(numerator)
    softmax = numerator/denominator
    return softmax
conf = softmax(choice_logits[0])[choices.index("A")]
print(conf)
pred = {0: "A", 1: "B", 2: "C", 3: "D"}[np.argmax(choice_logits[0])]
print(pred)